## Example 14 - Emission footprint of IWT

In [1]:
import opentnsim
print('This notebook has been tested with OpenTNSim version {}'.format(opentnsim.__version__))

This notebook has been tested with OpenTNSim version 1.0.0


In [2]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style
import folium

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from osgeo import ogr, osr

# OpenTNSIM
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot
import opentnsim.model as model

# Used for mathematical functions
import math             
import json

# Used for making the graph to visualize our problem
import networkx as nx  

# Graph location
location_graph = "Shape-Files/Vaarwegvakken"
name_graph = "Vaarwegvakken.shp"

# Vessel database
location_vessel_database = "Vessels\Vessel-database.csv"

### Initiate Simpy environment

In [3]:
# Start simpy environment
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))

### Load and transform graph

**Important**: 

If you use windows and get the following error "ImportError: read_shp requires OGR: http://www.gdal.org/", you probably have [this issue](https://github.com/conda-forge/gdal-feedstock/issues/219). Solving it is possible by running the following commands in your terminal (as explained [here](https://gis.stackexchange.com/questions/294231/installing-gdal-with-anaconda)):

```bash
#Create a new virtual environment
conda create -n testgdal -c conda-forge gdal vs2015_runtime=14

#Activate virtual environment
activate testgdal

#Open Jupyter notebook
jupyer notebook
```

In [4]:
FG = nx.read_shp(os.path.join(location_graph, name_graph), simplify=True)

# The read_shp creates a directed graph with single edges
# We require a directed graph but two-way traffic

FG = FG.to_undirected()
FG = FG.to_directed()

In [5]:
def transform_projection(location_graph, name_graph):
    driver = ogr.GetDriverByName("ESRI Shapefile")
    dataset = driver.Open(os.path.join(location_graph, name_graph))

    # from Layer
    inSpatialRef = dataset.GetLayer().GetSpatialRef()

    # Set up the coordinate reference we want to use, WGS84 - World Geodetic System 1984
    outSpatialRef = osr.SpatialReference()
    outSpatialRef.ImportFromEPSG(4326)

    # Transform the coordinates
    transform = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)
    
    return transform

In [6]:
def change_projection(transform, point):
    point = ogr.CreateGeometryFromWkt(str(point))
    
    point.Transform(transform)
    point.ExportToWkt()
    
    return point.GetX(), point.GetY()

In [7]:
transform = transform_projection(location_graph, name_graph)

FG_new = nx.DiGraph()
nodes_dict = {}

for i, node in enumerate(FG.nodes(data = True)):
    coordinates = change_projection(transform, shapely.geometry.Point(list(FG.nodes)[i][0], list(FG.nodes)[i][1]))
    name = "({:f}, {:f})".format(coordinates[0], coordinates[1])
    geometry = shapely.geometry.Point(coordinates[0], coordinates[1])
    
    nodes_dict[list(FG.nodes)[i]] = name
    FG_new.add_node(name, name = name, Position = coordinates, geometry = geometry, Old = node[1])
    
for edge in FG.edges(data = True):
    node_1 = nodes_dict[edge[0]]
    node_2 = nodes_dict[edge[1]]
    
    VRT_NAAM = edge[2]["VRT_NAAM"]
    VWG_NAAM = edge[2]["VWG_NAAM"]
    BEGKM =  edge[2]["BEGKM"]
    ENDKM =  edge[2]["ENDKM"]
    DIST = np.abs(float(BEGKM) - float(ENDKM))
    
    LINE = (json.loads(edge[2]["Json"])["coordinates"])
    LineString = []
    for coordinates in LINE:
        LineString.append(change_projection(transform, shapely.geometry.Point(coordinates[0], coordinates[1])))
    
    FG_new.add_edge(node_1, node_2, LineString = shapely.geometry.LineString(LineString), 
                    VRT_NAAM = VRT_NAAM, VWG_NAAM = VWG_NAAM, BEGKM = BEGKM, ENDKM = ENDKM, DIST = DIST)

if nx.info(FG) == nx.info(FG_new):
    print("Succes!")

Succes!


### Select only relevant area

In [8]:
# North-East
NE = (4.54, 51.75)
# South-East
SE = (4.54, 51.60)
# South-West
SW = (4.20, 51.60)
# North-West
NW = (4.20, 51.75)

polygon = shapely.geometry.Polygon([NE, SE, SW, NW])

In [9]:
nodes = []
edges = []

for edge in FG_new.edges(data = True):
    node_1 = FG_new.nodes[edge[0]]
    node_2 = FG_new.nodes[edge[1]]
    
    if node_1["geometry"].within(polygon) or node_2["geometry"].within(polygon):
        nodes.append(node_1)
        nodes.append(node_2)
        edges.append(edge)

In [10]:
FG_new = nx.DiGraph ()

for node in nodes:
    FG_new.add_node(node["name"], name = node["name"], Position = node["Position"], geometry = node["geometry"])

for edge in edges:
    FG_new.add_edge(edge[0], edge[1], Info = edge[2])

### Show on map

In [11]:
# Browser
m = folium.Map(location=[51.7, 4.4], zoom_start = 12)

for edge in FG_new.edges(data = True):
#     print(edge[2]["Info"]["VWG_NAAM"])
    points_x = list(edge[2]["Info"]["LineString"].coords.xy[0])
    points_y = list(edge[2]["Info"]["LineString"].coords.xy[1])
    
    line = []
    for i, _ in enumerate(points_x):
        line.append((points_y[i], points_x[i]))
    
    if edge[2]["Info"]["VWG_NAAM"] in ["Voorhavens Jachtensluis", "Voorhavens Volkeraksluizen"]:
        folium.PolyLine(line, color = "red", weight = 5, popup = edge[2]["Info"]["VWG_NAAM"]).add_to(m)
    
    else:
        folium.PolyLine(line, weight = 2, popup = edge[2]["Info"]["VWG_NAAM"]).add_to(m)

m

### Create locks
We can see on the maps that there are locks on the graph, but that the information on the locks is limited. The following edges represent locks (NB: click the edges to show info:

- Voorhavens Jachtensluis
- Voorhavens Volkeraksluizen

These edges will be replaced by two lock elements. The Jachtensluizen are mainly designed for yachts and pleasure craft, and have a length of 135 meters and a width of 16 meters. The Volkeraksluizen have three chambers each with a length of 330 meters and a width of 24 meters. For now we'll assume that both locks have a maximum allowable draught of 4.5 meters.

Additional information on the locks can be found on the [Rijkswaterstand website (link in Dutch)](https://www.rijkswaterstaat.nl/water/waterbeheer/bescherming-tegen-het-water/waterkeringen/deltawerken/volkeraksluizen.aspx). 

In [12]:
lock_nr_1 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 1", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_2 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 2", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_3 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 3", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

# lock_test = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Jachtensluis", 
#                         node_1 = "(4.395179, 51.691512)", node_2 = "(4.408442, 51.700226)",
#                         lock_length = 330, lock_width = 24, lock_depth = 4.5, 
#                         doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

In [13]:
for edge in FG_new.edges(data = True):
    if edge[2]["Info"]["VWG_NAAM"] == "Voorhavens Volkeraksluizen":
        # For testing, all locks have the water level at the right side
        lock_nr_1.water_level = "(4.430289, 51.700047)"
        lock_nr_2.water_level = "(4.430289, 51.700047)"
        lock_nr_3.water_level = "(4.430289, 51.700047)"
        
        # Add locks to the correct edge
        FG_new.edges[edge[0], edge[1]]["Lock"] = [lock_nr_1, lock_nr_2, lock_nr_3]

### Load vessel database

In [40]:
# NB: This database needs to be triple checked (!!!) ... ensure tracable sources
# consider use of Vessel database notebook
vessel_db = pd.read_csv(location_vessel_database)
vessel_db.head()

,VesselID,width,length,height_unloaded,height_loaded,draught_unloaded,draught_loaded,emissionfactor,installed_power,own_weight,...,capacity_unloaded,speed_loaded,speed_unloaded,resistance_loaded,resistance_unloaded,is_loaded,avv_class,cemt_class,type,scenario
0,29a24858-4a51-11e9-9792-b469212bff5b,5.05,38.5,2.0,5.25,1.5,2.7,0.73,36.0,80.0,...,0.0,3.083,4.470,16919.0,9188.0,0.0,M1,I,Spits,"['Base Case', 'NS-High', 'NS-Low']"
1,3254257e-4a51-11e9-8548-b469212bff5b,5.05,38.5,2.0,5.25,1.5,2.7,0.73,36.0,80.0,...,0.0,3.083,4.470,16919.0,9188.0,1.0,M1,I,Spits,"['Base Case', 'NS-High', 'NS-Low']"
2,dcc05ff4-4972-11e9-a543-b469212bff5b,6.06,55.0,2.0,6.10,1.5,2.7,0.72,118.0,90.0,...,0.0,3.361,4.638,28770.0,17524.0,0.0,M2,II,Kempenaar,"['Base Case', 'NS-High', 'NS-Low']"
3,dcc05ff4-4972-11e9-a543-b469212bff5b,6.06,55.0,2.0,6.10,1.5,2.7,0.72,118.0,90.0,...,0.0,3.361,4.638,28770.0,17524.0,1.0,M2,II,Kempenaar,"['Base Case', 'NS-High', 'NS-Low']"
4,e6cea50a-4972-11e9-a1f1-b469212bff5b,7.20,70.0,2.0,6.40,1.5,2.7,0.72,120.0,100.0,...,0.0,3.861,4.638,43935.0,22746.0,0.0,M3,III,Hagenaar,"['Base Case', 'NS-High', 'NS-Low']"


In [41]:
# renaming some of the vessel_db fields to align them with the code (NB: this alignment should be improved)
# in the end vessel_db should hold all (and not more than) the fields required by Vessel (see further down) 
vessel_db = vessel_db.rename(columns={
                                      'width': 'B',
                                      'length': 'L',
                                      'installed_power': 'P_installed',
                                      'capacity_loaded': 'capacity',
                                      'capacity_loaded': 'capacity',
                                      'height_unloaded': 'H_e', 
                                      'height_loaded': 'H_f', 
                                      'draught_unloaded': 'T_e', 
                                      'draught_loaded':  'T_f',
                                      'avv_class': 'type'})

# remove parameters that are not used (later we may want to add these via a mix-in)
del vessel_db['VesselID']
del vessel_db['own_weight']
del vessel_db['capacity_unloaded']
del vessel_db['speed_loaded']
del vessel_db['speed_unloaded']
del vessel_db['is_loaded']
del vessel_db['cemt_class']
del vessel_db['scenario']
del vessel_db['resistance_loaded']
del vessel_db['resistance_unloaded']
del vessel_db['emissionfactor']

In [42]:
vessel_db['C_b'] = 0.922218379546862
vessel_db['C_BB'] = 0.185

In [43]:
vessel_db.head()

,B,L,H_e,H_f,T_e,T_f,P_installed,capacity,type,type,C_b,C_BB
0,5.05,38.5,2.0,5.25,1.5,2.7,36.0,260.0,M1,Spits,0.922218,0.185
1,5.05,38.5,2.0,5.25,1.5,2.7,36.0,260.0,M1,Spits,0.922218,0.185
2,6.06,55.0,2.0,6.10,1.5,2.7,118.0,422.5,M2,Kempenaar,0.922218,0.185
3,6.06,55.0,2.0,6.10,1.5,2.7,118.0,422.5,M2,Kempenaar,0.922218,0.185
4,7.20,70.0,2.0,6.40,1.5,2.7,120.0,520.0,M3,Hagenaar,0.922218,0.185


In [44]:
class VesselProperties:
    """Mixin class: Something that has vessel properties

    type: can contain info on vessel type (avv class, cemt_class or other)
    B: vessel width
    L: vessel length
    H_e: vessel height unloaded
    H_f: vessel height loaded
    T_e: draught unloaded
    T_f: draught loaded

    Add information on possible restrictions to the vessels, i.e. height, width, etc.
    """

    def __init__(
        self,
        type,
        B,
        L,
        H_e,
        H_f,
        T_e,
        T_f,
        *args,
        **kwargs
        ):
        super().__init__(*args, **kwargs)

        """Initialization"""
        self.type = type

        self.B = B
        self.L = L

        self.H_e = H_e
        self.H_f = H_e

        self.T_e = T_e
        self.T_f = T_f

    @property
    def H(self):
        """ Calculate current height based on filling degree """

        return (
            self.filling_degree * (self.H_f - self.H_e)
            + self.H_e
        )

    @property
    def T(self):
        """ Calculate current draught based on filling degree
        
        Here we should implement the rules from Van Dorsser et al 
        """

        return (
            self.filling_degree * (self.T_f - self.T_e)
            + self.T_e
        )

    def get_route(
        self,
        origin,
        destination,
        graph=None,
        minWidth=None,
        minHeight=None,
        minDepth=None,
        randomSeed=4,
    ):
        """ Calculate a path based on vessel restrictions """

        graph = graph if graph else self.env.FG
        minWidth = minWidth if minWidth else 1.1 * self.B
        minHeight = minWidth if minHeight else 1.1 * self.H
        minDepth = minWidth if minDepth else 1.1 * self.T

        # Check if information on restrictions is added to the edges
        random.seed(randomSeed)
        edge = random.choice(list(graph.edges(data=True)))
        edge_attrs = list(edge[2].keys())

        # IMPROVE THIS TO CHECK ALL EDGES AND COMBINATIONS OF RESTRICTIONS

        if all(item in edge_attrs for item in ["Width", "Height", "Depth"]):
            edges = []
            nodes = []

            for edge in graph.edges(data=True):
                if (
                    edge[2]["Width"] >= minWidth
                    and edge[2]["Height"] >= minHeight
                    and edge[2]["Depth"] >= minDepth
                ):
                    edges.append(edge)

                    nodes.append(graph.nodes[edge[0]])
                    nodes.append(graph.nodes[edge[1]])

            subGraph = graph.__class__()

            for node in nodes:
                subGraph.add_node(
                    node["name"],
                    name=node["name"],
                    geometry=node["geometry"],
                    position=(node["geometry"].x, node["geometry"].y),
                )

            for edge in edges:
                subGraph.add_edge(edge[0], edge[1], attr_dict=edge[2])

            try:
                return nx.dijkstra_path(subGraph, origin, destination)
            except:
                raise ValueError(
                    "No path was found with the given boundary conditions."
                )

        # If not, return shortest path
        else:
            return nx.dijkstra_path(graph, origin, destination)


In [45]:
class HasEnergy:
    """Mixin class: Something that has energy usage.

    P_installed: installed engine power [kW]
    C_b: blockage factor [-]
    C_BB: breadth coefficient [-]
    nu: kinematic viscosity [m^2/s]
    rho: density of the surrounding water [kg/m^3]
    g: gravitational accelleration [m/s^2]
    x: number of propellors [-]
    eta_0: open water efficiency of propellor [-]
    eta_r: relative rotative efficiency [-]
    eta_t: transmission efficiency [-]
    eta_g: gearing efficiency [-]
    c_stern: shape of the afterbody [-]
    one_k2: appendage resistance factor [-]
    """

    def __init__(
        self, 
        P_installed,
        C_b,
        C_BB,
        nu = 1*10**(-6), # kinematic viscosity
        rho = 1000,
        g = 9.81,
        x = 2, # number of propellors
        eta_0 = 0.7,
        eta_r = 0.98,
        eta_t = 0.98,
        eta_g = 0.96,
        c_stern = 0,
        one_k2 = 1.5,
        *args, 
        **kwargs
        ):
        super().__init__(*args, **kwargs)

        """Initialization"""
        self.P_installed = P_installed
        self.C_b = C_b
        self.C_BB = C_BB
        self.nu = nu 
        self.rho = rho
        self.g = g
        self.x = x
        self.eta_0 = eta_0
        self.eta_r = eta_r
        self.eta_t = eta_t
        self.eta_g = eta_g
        self.c_stern = c_stern
        self.one_k2 = one_k2
                
    def calculate_properties(self):
        self.C_M = 1.006 - 0.0056 * self.C_b ** (-3.56)
        self.C_wp = (1 + 2 * self.C_b) / 3
        self.C_p = self.C_b / self.C_M

        self.delta = self.C_b * self.L * self.B * self.T #water displacement

        self.lcb = -13.5 + 19.4 * self.C_p
        self.L_R = self.L * (1 - self.C_p + (0.06 * self.C_p * self.lcb)/(4 * self.C_p - 1))

        self.A_BT = self.C_BB * self.B * self.T * self.C_M
        self.A_T = 0.1 * self.B * self.T

        self.S_T = self.L * (2 * self.T + self.B) * np.sqrt(self.C_M) * (0.453 + 0.4425 * self.C_b - 0.2862 * self.C_M - 0.003467 * (self.B / self.T) + 0.3696 * self.C_wp) + 2.38 * (self.A_BT / self.C_b)
        self.S_APP = 0.05 * self.S_T
        self.S_B = self.L * self.B

        self.D_s = 0.7 * self.T
        
    def calculate_frictional_resistance(self, V_0, h):
        """Section 2.2.1 Frictional resistance"""

        self.R_e = V_0 * self.L / self.nu
        self.D = h - self.T #distance from bottom ship to the bottom of the fairway

        self.Cf_0 = 0.075 / ((np.log10(self.R_e) - 2) ** 2)
        self.Cf_proposed = (0.08169 / ((np.log10(self.R_e) - 1.717) ** 2)) * (1 + (0.003998 / (np.log10(self.R_e) - 4.393)) * (self.D / self.L) ** (-1.083))

        self.a = 0.042612 * np.log10(self.R_e) + 0.56725
        self.Cf_katsui = 0.0066577 / ((np.log10(self.R_e) - 4.3762) ** self.a)

        self.V_B = 0.4277 * V_0 * np.exp((h / self.T) ** (-0.07625))
                
        self.C_f = self.Cf_0 + (self.Cf_proposed - self.Cf_katsui) * (self.S_B / self.S_T) * (self.V_B / V_0) ** 2

        self.R_f = (self.C_f * 0.5 * self.rho * (V_0 ** 2) * self.S_T) / 1000 #kN

    def calculate_viscous_resistance(self):
        """Section 2.2.2 Viscous resistance"""

        self.c_14 = 1 + 0.0011 * self.c_stern
        self.one_k1 = 0.93 + 0.487 * self.c_14 * ((self.B / self.L) ** 1.068) * ((self.T/self.L) ** 0.461) * ((self.L / self.L_R) ** 0.122) * (((self.L ** 3) / self.delta) ** 0.365) * ((1 - self.C_p) ** (-0.604))
        
    def calculate_appendage_resistance(self, V_0):
        """Section 2.2.3 Appendage resistance"""
        
        self.R_APP = (0.5 * self.rho * (V_0 ** 2) * self.S_APP * self.one_k1 * self.C_f) / 1000 #kN
     
    def calculate_wave_resistance(self, V_0):
        """Section 2.2.4 Wave resistance"""

        self.F_n = V_0 / np.sqrt(self.g * self.L) #Froude number

        #coefficient c_7
        if self.B / self.L < 0.11:
            self.c_7 = 0.229577 * (self.B / self.L) ** 0.33333
        elif self.B / self.L > 0.25:
            self.c_7 = 0.5 - 0.0625 * (self.L / self.B)
        else:
            self.c_7 = self.B / self.L

        self.i_E = 125.67 * (self.B / self.L) - 162.25 * (self.C_p ** 2) + 234.32 * (self.C_p ** 3) + 0.155087 * (self.lcb ** 3)
        self.c_1 = 2223105 * (self.c_7 ** 3.78613) * ((self.T / self.B) ** 1.07961) * ((90 - self.i_E) ** (-1.37565))

        self.c_3 = 0.56 * (self.A_BT ** 1.5) / (self.B * self.T * (0.31 * np.sqrt(self.A_BT) + self.T - 0.5*self.T))
        self.c_2 = np.exp(-1.89 * np.sqrt(self.c_3))
        self.c_5 = 1 - 0.8 * self.A_T / (self.B * self.T * self.C_M)

        #coefficient c_16
        if self.C_p < 0.80:
            self.c_16 = 8.07981 * self.C_p - (13.8673 * self.C_p ** 2) + (6.984388 * self.C_p ** 3)
        else:
            self.c_16 = 1.73014 - 0.7067 * self.C_p

        self.m_1 = 0.0140407 * (self.L/self.T) - (1.75254 * self.delta**(1/3))/self.L - 4.79323 * (self.B/self.L) - self.c_16

        #coefficient c_15
        if (self.L**3)/self.delta < 512:
            self.c_15 = -1.69385
        elif (self.L**3)/self.delta > 1727:
            self.c_15 = 0
        else:
            self.c_15 = -1.69385 + (self.L / (self.delta**(1/3)) - 8.0) / 2.36

        self.m_2 = self.c_15 * (self.C_p**2) * np.exp(-0.1 * (self.F_n)**(-2))

        #coefficient lambda
        if self.L/self.B < 12:
            self.lmbda = 1.446 * self.C_p - 0.03 * (self.L/self.B)
        else:
            self.lmbda = 1.446 * self.C_p - 0.36


        self.R_W = (self.c_1 * self.c_2 * self.c_5 * self.delta * self.rho * self.g * np.exp(self.m_1 * (self.F_n**-0.9) + self.m_2 * np.cos(self.lmbda * (self.F_n**(-2))))) / 1000 #kN

    def calculate_residual_resistance(self, V_0):
        """Section 2.2.5 Residual resistance terms"""

        #Resistance resulting from the bulbouw bow
        self.P_B = 0.56 * np.sqrt(self.A_BT) / (self.T - 1.5 * 0.5*self.T)
        self.F_ni = V_0 / np.sqrt(self.g * (self.T - 0.5*self.T - 0.25*np.sqrt(self.A_BT)) + 0.15*(V_0**2))

        self.R_B = (0.11 * np.exp(-3 * self.P_B**(-2)) * (self.F_ni**3) * (self.A_BT**1.5) * self.rho * self.g / (1 + (self.F_ni**2))) / 1000 #kN

        #Resistance due to immersed transom
        self.F_nt = V_0 / np.sqrt(2 * self.g * self.A_T / (self.B + self.B * self.C_wp))

        # ToDo: didn't undertand what happened here. Think this was for running with an array of values?
        self.c_6 = 0.2 * (1 - 0.2 * self.F_nt)

#         self.c_6 = np.zeros(101)
#         counter = 0 
#         for i in self.F_nt:
#             if i < 5:
#                 self.c_6[counter] = 0.2 * (1 - 0.2 * i)
#             else:
#                 self.c_6[counter] = 0

#             counter += 1

        self.R_TR = (0.5 * self.rho * (V_0**2) * self.A_T * self.c_6) / 1000 #kN

        #Model-ship correlation resistance
        if self.T/self.L > 0.04:
            self.c_4 = 0.04
        else:
            self.c_4 = self.T / self.L

        self.C_A = 0.006 * ((self.L + 100) ** (-0.16)) - 0.00205 + 0.003 * np.sqrt(self.L / 7.5) * (self.C_b ** 4) * self.c_2 * (0.04 - self.c_4)

        self.R_A = (0.5 * self.rho * (V_0 ** 2) * self.S_T * self.C_A) / 1000 #kW

    def calculate_total_resistance(self, V_0, h):
        
        self.calculate_properties()
        self.calculate_frictional_resistance(V_0, h)
        self.calculate_viscous_resistance()
        self.calculate_appendage_resistance(V_0)
        self.calculate_wave_resistance(V_0)
        self.calculate_residual_resistance(V_0)
        
        self.R_tot = self.R_f * self.one_k1 + self.R_APP + self.R_W + self.R_B + self.R_TR + self.R_A

    def calculate_total_power_required(self):
        """ Section 2.1 Total required power """

        #2.1.1 Required power for systems on board
        self.P_hotel = 0.081 * self.P_installed

        #2.1.2 Required power for propulsion

        #Effective Horse Power (EHP)
        self.P_EHP = self.V_B * self.R_tot

        dw = np.zeros(101)
        counter = 0 

        #Calculation hull efficiency
        if self.F_n < 0.2:
            self.dw = 0
        else:
            self.dw = 0.1


        self.w = 0.11 * (0.16 / self.x) * self.C_b * np.sqrt((self.delta**(1/3)) / self.D_s) - self.dw

        if self.x == 1:
            self.t = 0.6 * self.w * (1 + 0.67 * self.w)
        else:
            self.t = 0.8 * self.w * (1 + 0.25 * self.w)

        self.eta_h = (1 - self.t) / (1 - self.w)

        #Delivered Horse Power (DHP)

        self.P_DHP = self.P_EHP / (self.eta_0 * self.eta_r * self.eta_h)

        #Brake Horse Power (BHP)
        self.P_BHP = self.P_DHP / (self.eta_t * self.eta_g)

        self.P_tot = self.P_hotel + self.P_BHP

In [46]:
# Vessel type
Vessel = type('Vessel', 
              (core.Identifiable,      # so we can identify an individual vessel (by name and id)
               VesselProperties,       # so that we can provide vessel properties
               core.Movable,           # so we van make the vessel move
               core.HasContainer,      # so that we can indicate the amount of cargo it carries
               core.HasResource,       # so that a vessel be requested to do something
               core.Routeable,         # so that you can provide a route that the vessel will follow
               HasEnergy),        # so that you can calculate energy used on route
              {})

In [47]:
vessel = Vessel(name='Vessel', v=1, type=[], 
                geometry=[], 
                capacity=1, level=1,
                B=11.75, L = 135, 
                H_e=[], H_f=[], T_e=2.75, T_f=2.75, 
                route=[], env=[],
                P_installed=1267,
                C_b = 0.922218379546862,
                C_BB = 0.185
                )

In [48]:
generator = model.VesselGenerator(Vessel, vessel_db)

### Run simulation

In [49]:
# NB simpy environment was already initiated 
env.FG = FG_new

In [50]:
# Randomly draw a number of vessels (with random origins and destinations)
vessels = []

# Add 10 vessels to the simulation
for i in range(10):
    random_1 = random.choice(list(env.FG))
    random_2 = random.choice(list(env.FG))
    path = nx.dijkstra_path(env.FG, random_1, random_2)
    
    vessel = generator.generate(env, "Vessel " + str(i))
    vessel.route = path
    vessel.geometry = nx.get_node_attributes(env.FG, "geometry")[vessel.route[0]]
    vessels.append(vessel)
    
    # Add the movements of the vessel to the simulation
    env.process(vessel.move())
#     env.process(start(env, vessel))

In [51]:
# Run simulation
env.run()

### Check results

In [52]:
# pick a vessel from the vessels list and inspect the results (for now it does not seem the locks are working)
for vessel in vessels:
    print(vessel.name)
    df = pd.DataFrame.from_dict(vessel.log)

    # print all messages
    for index, row in df.iterrows():
        print(row['Message'])

Vessel 0
Sailing from node (4.415027, 51.704766) to node (4.423482, 51.710607) start
Sailing from node (4.415027, 51.704766) to node (4.423482, 51.710607) start
Sailing from node (4.423482, 51.710607) to node (4.436174, 51.702279) start
Sailing from node (4.423482, 51.710607) to node (4.436174, 51.702279) start
Sailing from node (4.436174, 51.702279) to node (4.430289, 51.700047) start
Sailing from node (4.436174, 51.702279) to node (4.430289, 51.700047) start
Passing lock start
Passing lock stop
Vessel 1
Sailing from node (4.383376, 51.716016) to node (4.401283, 51.713412) start
Sailing from node (4.383376, 51.716016) to node (4.401283, 51.713412) start
Sailing from node (4.401283, 51.713412) to node (4.423482, 51.710607) start
Sailing from node (4.401283, 51.713412) to node (4.423482, 51.710607) start
Sailing from node (4.423482, 51.710607) to node (4.415027, 51.704766) start
Sailing from node (4.423482, 51.710607) to node (4.415027, 51.704766) start
Sailing from node (4.415027, 51.7

In [54]:
V_0 = 4 # m/s
h = 10  # m

vessels[0].calculate_total_resistance(V_0, h)
vessels[0].calculate_total_power_required()

print(vessels[0].R_tot)
print(vessels[0].P_tot)

324.84457962687117
2064.204438850243


In [55]:
vessels[0].__dict__

{'P_installed': 380.0,
 'C_b': 0.922218379546862,
 'C_BB': 0.185,
 'nu': 1e-06,
 'rho': 1000,
 'g': 9.81,
 'x': 2,
 'eta_0': 0.7,
 'eta_r': 0.98,
 'eta_t': 0.98,
 'eta_g': 0.96,
 'c_stern': 0,
 'one_k2': 1.5,
 'env': <simpy.core.Environment at 0x1561442beb0>,
 'log': {'Message': ['Sailing from node (4.415027, 51.704766) to node (4.423482, 51.710607) start',
   'Sailing from node (4.415027, 51.704766) to node (4.423482, 51.710607) start',
   'Sailing from node (4.423482, 51.710607) to node (4.436174, 51.702279) start',
   'Sailing from node (4.423482, 51.710607) to node (4.436174, 51.702279) start',
   'Sailing from node (4.436174, 51.702279) to node (4.430289, 51.700047) start',
   'Sailing from node (4.436174, 51.702279) to node (4.430289, 51.700047) start',
   'Passing lock start',
   'Passing lock stop'],
  'Timestamp': [datetime.datetime(2020, 7, 25, 8, 48, 54),
   datetime.datetime(2020, 7, 25, 9, 3, 28, 4915),
   datetime.datetime(2020, 7, 25, 9, 3, 28, 4915),
   datetime.datetim

### Calculate energy use

In [23]:
# Define a EnergyCalculation class (this class postprocesses all results and calculates the energy (kWh))
class EnergyCalculation:
    """
    Add information on energy use and effects on energy use.
    """

    def __init__(self, vessel, *args, **kwargs):
        super().__init__(*args, **kwargs)

        """Initialization"""
        self.vessel = vessel
        self.emissionfactor = self.vessel.emissionfactor
        self.energy_use = {"time_start": [], 
                           "time_stop": [], 
                           "edge_start": [],
                           "edge_stop": [],
                           "total_energy": [], 
                           "stationary": []}
        self.co2_footprint = {"total_footprint": 0, "stationary": 0}
        self.mki_footprint = {"total_footprint": 0, "stationary": 0}

    @property
    def power(self):
        # I think this is rougly Eq 2.20 from Vehmeijer (2009), and exactly Eq 15 from Bolt (2003)
        return 2 * (self.vessel.current_speed * self.vessel.resistance * 10 ** -3)  # kW

    def calculate_energy_consumption(self):
        """Calculation of energy consumption based on total time in system and properties"""

        stationary_phase_indicator = [
            "Waiting to enter waiting area stop",
            "Waiting in waiting area stop",
            "Waiting in line-up area stop",
            "Passing lock stop",
        ]
        

        times = self.vessel.log["Timestamp"]
        messages = self.vessel.log["Message"]
        geometries = self.vessel.log["Geometry"]
        for i in range(len(times) - 1):
            delta_t = (times[i + 1] - times[i]).seconds
            if delta_t != 0:
                self.energy_use["time_start"].append(times[i])
                self.energy_use["time_stop"].append(times[i + 1])
                self.energy_use["edge_start"].append(geometries[i])
                self.energy_use["edge_stop"].append(geometries[i + 1])

                if messages[i + 1] in stationary_phase_indicator:
                    # Hier wordt 15% van de energy_delta genomen, voor stationaire fases
                    energy_delta = self.power * delta_t / 3600  # kJ/3600 = kWh
                    self.energy_use["total_energy"].append(energy_delta * 0.15)
                    self.energy_use["stationary"].append(energy_delta * 0.15)

                else:
                    # Hier wordt de energie genomen van varen op kruissnelheid 
                    energy_delta = self.power * delta_t / 3600  # kJ/3600 = kWh
                    self.energy_use["total_energy"].append(energy_delta)
                    self.energy_use["stationary"].append(0)

        # TODO: er moet hier een heel aantal dingen beter worden ingevuld
        # - de kruissnelheid is nu nog per default 1 m/s (zie de Movable mixin). Eigenlijk moet in de 
        #   vessel database ook nog een speed_loaded en een speed_unloaded worden toegevoegd. 
        # - de resistance factor (Rt) moet eigenlijk voor elke vaarweg sectie worden uitgerekend.
        # - er zou nog eens goed gekeken moeten worden wat er gedaan kan worden rond kustwerken 
        #   (nu dus 15% van kruissnelheid, wellicht is er iets mogelijk als wat Vibeke van der Bilt
        #    had gedaan. Die keek naar bronnen waar de energie aan opging: zoveel procent voortstuwing,
        #    zoveel procent boornet, etc.)

In [30]:
vessels[nr].__dict__

{'installed_power': 160.0,
 'resistance': 50582.0,
 'resistance_empty': 28777.0,
 'emissionfactor': 0.72,
 'env': <simpy.core.Environment at 0x1375d04ae50>,
 'log': {'Message': ['Sailing from node (4.622926, 51.624686) to node (4.517077, 51.607229) start',
   'Sailing from node (4.622926, 51.624686) to node (4.517077, 51.607229) start',
   'Sailing from node (4.517077, 51.607229) to node (4.482176, 51.619954) start',
   'Sailing from node (4.517077, 51.607229) to node (4.482176, 51.619954) start',
   'Sailing from node (4.482176, 51.619954) to node (4.436117, 51.618258) start',
   'Sailing from node (4.482176, 51.619954) to node (4.436117, 51.618258) start',
   'Sailing from node (4.436117, 51.618258) to node (4.427668, 51.625182) start',
   'Sailing from node (4.436117, 51.618258) to node (4.427668, 51.625182) start',
   'Sailing from node (4.427668, 51.625182) to node (4.370807, 51.657423) start',
   'Sailing from node (4.427668, 51.625182) to node (4.370807, 51.657423) start',
   'S

In [24]:
# Instantiate an EnergyCalculation object for a particular vessel
# Change nr to look at a different vessel
nr=2
energycalculation = EnergyCalculation(vessels[nr])

# Calculate the energy use based on the vessel log information
energycalculation.calculate_energy_consumption()

# Show the output in a dataframe
df = pd.DataFrame.from_dict(energycalculation.energy_use)
print('{}: Total energy used in this trip is {:.2f} kWh'.format(vessels[nr].name, np.sum(df["total_energy"])))
df

Vessel 2: Total energy used in this trip is 701.52 kWh


,time_start,time_stop,edge_start,edge_stop,total_energy,stationary
0,2020-07-24 15:30:04.000000,2020-07-24 17:36:28.413553,POINT (4.62292558877379 51.62468615267751),POINT (4.517077105310467 51.60722923324587),213.118827,0.00000
1,2020-07-24 17:36:28.413553,2020-07-24 18:23:09.969629,POINT (4.517077105310467 51.60722923324587),POINT (4.482175776949589 51.61995356683045),78.711212,0.00000
2,2020-07-24 18:23:09.969629,2020-07-24 19:16:25.534264,POINT (4.482175776949589 51.61995356683045),POINT (4.436117162846041 51.61825782350502),89.783050,0.00000
3,2020-07-24 19:16:25.534264,2020-07-24 19:32:32.997912,POINT (4.436117162846041 51.61825782350502),POINT (4.427667577930265 51.62518233952007),27.173774,0.00000
4,2020-07-24 19:32:32.997912,2020-07-24 21:01:18.472869,POINT (4.427667577930265 51.62518233952007),POINT (4.370807285850249 51.65742268295148),149.638417,0.00000
5,2020-07-24 21:01:18.472869,2020-07-24 21:04:33.152254,POINT (4.370807285850249 51.65742268295148),POINT (4.368885256028107 51.65870041326348),5.451616,0.00000
6,2020-07-24 21:04:33.152254,2020-07-24 21:16:29.985997,POINT (4.368885256028107 51.65870041326348),POINT (4.362481897892667 51.66376506070044),20.120396,0.00000
7,2020-07-24 21:16:29.985997,2020-07-24 21:57:03.744049,POINT (4.362481897892667 51.66376506070044),POINT (4.388034316357048 51.67880024954439),68.370003,0.00000
8,2020-07-24 21:57:03.744049,2020-07-24 22:03:58.622243,POINT (4.388034316357048 51.67880024954439),POINT (4.392555365264324 51.68125080672722),11.633860,0.00000
9,2020-07-24 22:03:58.622243,2020-07-24 22:48:58.622243,POINT (4.62292558877379 51.62468615267751),POINT (4.430289385790487 51.70004667570937),11.380950,11.38095


### Show loginfo of the locks

In [25]:
pd.DataFrame.from_dict(lock_nr_1.log)

,Message,Timestamp,Value,Geometry
0,Lock doors closing start,2020-07-24 16:13:50.647430,"(4.430289, 51.700047)",0
1,Lock doors closing stop,2020-07-24 16:23:50.647430,"(4.430289, 51.700047)",0
2,Lock chamber converting start,2020-07-24 16:23:50.647430,"(4.430289, 51.700047)",0
3,Lock chamber converting stop,2020-07-24 16:48:50.647430,"(4.392555, 51.681251)",0
4,Lock doors opening start,2020-07-24 16:48:50.647430,"(4.392555, 51.681251)",0
5,Lock doors opening stop,2020-07-24 16:58:50.647430,"(4.392555, 51.681251)",0
6,Lock doors closing start,2020-07-24 22:03:58.622243,"(4.392555, 51.681251)",0
7,Lock doors closing stop,2020-07-24 22:13:58.622243,"(4.392555, 51.681251)",0
8,Lock chamber converting start,2020-07-24 22:13:58.622243,"(4.392555, 51.681251)",0
9,Lock chamber converting stop,2020-07-24 22:38:58.622243,"(4.430289, 51.700047)",0


In [26]:
pd.DataFrame.from_dict(lock_nr_2.log)

,Message,Timestamp,Value,Geometry
0,Lock doors closing start,2020-07-24 16:28:42.580584,"(4.430289, 51.700047)",0
1,Lock doors closing stop,2020-07-24 16:38:42.580584,"(4.430289, 51.700047)",0
2,Lock chamber converting start,2020-07-24 16:38:42.580584,"(4.430289, 51.700047)",0
3,Lock chamber converting stop,2020-07-24 17:03:42.580584,"(4.392555, 51.681251)",0
4,Lock doors opening start,2020-07-24 17:03:42.580584,"(4.392555, 51.681251)",0
5,Lock doors opening stop,2020-07-24 17:13:42.580584,"(4.392555, 51.681251)",0


In [27]:
pd.DataFrame.from_dict(lock_nr_3.log)

,Message,Timestamp,Value,Geometry


### Visualise on Google Earth

In [28]:
plot.vessel_kml(env, vessels)

### Sandbox

In [31]:
vessels[0].__dict__

{'installed_power': 380.0,
 'resistance': 167863.0,
 'resistance_empty': 102290.0,
 'emissionfactor': 0.7,
 'env': <simpy.core.Environment at 0x1375d04ae50>,
 'log': {'Message': ['Sailing from node (4.415027, 51.704766) to node (4.423482, 51.710607) start',
   'Sailing from node (4.415027, 51.704766) to node (4.423482, 51.710607) start',
   'Sailing from node (4.423482, 51.710607) to node (4.436174, 51.702279) start',
   'Sailing from node (4.423482, 51.710607) to node (4.436174, 51.702279) start',
   'Sailing from node (4.436174, 51.702279) to node (4.430289, 51.700047) start',
   'Sailing from node (4.436174, 51.702279) to node (4.430289, 51.700047) start',
   'Passing lock start',
   'Passing lock stop'],
  'Timestamp': [datetime.datetime(2020, 7, 24, 15, 30, 4),
   datetime.datetime(2020, 7, 24, 15, 44, 38, 4915),
   datetime.datetime(2020, 7, 24, 15, 44, 38, 4915),
   datetime.datetime(2020, 7, 24, 16, 5, 54, 23426),
   datetime.datetime(2020, 7, 24, 16, 5, 54, 23426),
   datetime